In [1]:
from numpy import *
import operator

#进行数据准备
labels=['A','A','B','B']
dataSet=array([[1.0,1.1],[1.0,1.0],[0.0,0.0],[0.1,0.0]])


In [2]:
def classify0(inx,dataSet,labels,k):
    dataSetSize = dataSet.shape[0]
    #inx初始是1行的数组，通过tile函数进行数组广播，把数组变成行数和dataSet行数相等
    #对应位置相减之后通过平方测量距离
    diffMat = tile(inx,(dataSetSize,1))-dataSet
    sqDiffMat = diffMat ** 2 #平方距离
    sqDistance = sqDiffMat.sum(axis=1)  #axis=0列　axis=1横
    distances = sqDistance ** 0.5  #开根号
    sortedDistances = distances.argsort()  #numpy.argsort() 函数返回的是数组值从小到大的索引值。

    classCount = {}
    #依据排序顺序从近到远依次查看ｋ个样例，记录类别和数量
    for i in range (k):
        voteLebel = labels[sortedDistances[i]]
        classCount[voteLebel]=classCount.get(voteLebel,0)+1  #(key,dafault) 如果key不存在，返回默认值

    #对记录类别数量按数量从大到小排序,并且将字典分解为元组列表
    #dict.items()返回可遍历的(键, 值) 元组数组。operator.itemgetter(1)代表按照键值对中的值进行排序．reverse=True从大到小排序
    sortedClassCount = sorted(classCount.items(),key=operator.itemgetter(1),reverse=True)
  
    return sortedClassCount[0][0]

In [3]:
def file2matrix(filename):
    fr = open(filename)#打开文件，返回对象
    arrayOLines = fr.readlines()
    numberOfLines = len(arrayOLines)#得到文件行数
    returnMat = zeros((numberOfLines,3))#全0的矩阵，行是文件行数，列是3
    classLabelVector = [] #标签存在元组中
    index = 0
    for line in arrayOLines:
        line = line.strip()#Python strip() 方法用于移除字符串头尾指定的字符（默认为空格）或字符序列。不能删除中间部分的字符。
        listFromLine = line.split('\t') #split() 通过指定分隔符对字符串进行切片,此处是回车
        returnMat[index,:] = listFromLine[0:3] #将从文件中读取到的数据存放在矩阵中
        classLabelVector.append(int(listFromLine[-1]))
        index+=1
    fr.close()
    return returnMat,classLabelVector

In [4]:
datingDataMat,datingLabels=file2matrix('datingTestSet2.txt')


In [5]:
import matplotlib
import matplotlib.pyplot as plt
fig = plt.figure()
#add_subplot(349)参数349的意思是：将画布分割成3行4列，图像画在从左到右从上到下的第9块，当我们只想画一副图的时候，使用参数‘111’即可。
ax = fig.add_subplot(111) #subplot() 函数允许你在同一图中绘制不同的东西。
#scatter把点呈现出来　scatter函数的参数如下
#def scatter(x, y, s=None, c=None, marker=None, cmap=None, norm=None, vmin=None, vmax=None, verts=None, 
#edgecolors=None, hold=None, data=None, **kwargs) x,y是数据　ｓ是大小　ｃ是颜色

#黄色极具魅力，蓝色一般，紫色不喜欢
#ax.scatter(datingDataMat[:,1],datingDataMat[:,2],15.0*array(datingLabels),15.0*array(datingLabels))
#plt.show()
ax.scatter(datingDataMat[:,0],datingDataMat[:,1],15.0*array(datingLabels),15.0*array(datingLabels))
plt.show()

<Figure size 640x480 with 1 Axes>

In [6]:
def autoNorm(dataSet):
    minVals=dataSet.min(0)#0是列，1是行
    maxVals=dataSet.max(0)
    ranges=maxVals-minVals
    m=dataSet.shape[0]
    normDataSet=zeros(shape(dataSet)) 
    normDataSet=dataSet-tile(minVals,(m,1))
    normDataSet=normDataSet/tile(ranges,(m,1))
    return normDataSet,ranges,minVals

In [7]:
normMat,ranges,minVals=autoNorm(datingDataMat)
print(normMat)

[[0.44832535 0.39805139 0.56233353]
 [0.15873259 0.34195467 0.98724416]
 [0.28542943 0.06892523 0.47449629]
 ...
 [0.29115949 0.50910294 0.51079493]
 [0.52711097 0.43665451 0.4290048 ]
 [0.47940793 0.3768091  0.78571804]]


In [8]:
def datingClassTest():
    hoRatio = 0.1
    datingDataMat,datingLabels = file2matrix('datingTestSet2.txt') #读取文件内容到矩阵
    normMat,ranges,minVals = autoNorm(datingDataMat)#将矩阵标准化，让每个特征都在0-1之间
    m = normMat.shape[0]#m是矩阵行数，即样例个数
    testNum = int(m*hoRatio)#测试的个数
    errorNum = 0.0#累计判断错误的个数
    for i in range(testNum):
        testResult = classify0(normMat[i,:],normMat[testNum:m,:],datingLabels[testNum:m],3)
        print("the classifyier came back with:%d,the real answer is:%d" %(testResult,datingLabels[i]))
        if(testResult!=datingLabels[i]):
            errorNum+=1
    print("the total error rate is:%d" % (errorNum/float(testNum)))
    

In [9]:
#datingClassTest()

In [10]:
def classifyPerson():
    result=['not at all','a little like','very like']
    #读入待测试数据
    ffMile = float(input("flier miles per year?"))
    percentTats = float(input("pencentage of time spent playing video game?"))
    iceCream = float(input("liters of ice cream consumed per year?"))
    testMat = [ffMile,percentTats,iceCream]#测试矩阵
    datingDataMat,datingLabels = file2matrix('datingTestSet2.txt')
    normMat,ranges,minVals = autoNorm(datingDataMat)
    testResult = classify0(((testMat-minVals)/ranges),normMat,datingLabels,3)#得到测试结果
    print("test person result:", (result[testResult-1]))
    

In [11]:
#classifyPerson()

In [12]:
def img2vector(filename):#将文件读入到一维vector中，txt文件是32*32,读入为1*1024
    returnVect = zeros((1,1024)) 
    fr = open(filename)
    for i in range (32):
        linestr = fr.readline()
        for j in range (32):
            returnVect[0,j+i*32]=int(linestr[j])
    return returnVect
    

In [18]:
testVector = img2vector('testDigits/0_13.txt')
#print(testVector[0,0:32])

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0.]


In [52]:
def handwritingClassTest():
    hwLabels = []
    #os.listdir() 方法用于返回指定的文件夹包含的文件或文件夹的名字的列表。这个列表以字母顺序。 它不包括 '.' 和'..' 即使它在文件夹中。
    trainingFileList = listdir('trainingDigits')#读入顺序和原来文件排列顺序不一定一致
    m = len(trainingFileList)
    trainingMat = zeros((m,1024))
    for i in range(m):
        fileName = trainingFileList[i]
        name = fileName.split('.')[0]
        label = name.split('_')[0]
        hwLabels.append(label)
        trainingMat[i,:] = img2vector('trainingDigits/%s' %fileName)

    error = 0.0
    testFileList = listdir('testDigits')
    n = len(testFileList)
    testMat = zeros((1,1024))
    for j in range(n):
        testFileName = testFileList[j]
        testName = testFileName.split('.')[0]
        testLabel = int(testName.split('_')[0])
        testMat = img2vector('testDigits/%s' % testFileName)
        result = int(classify0(testMat,trainingMat,hwLabels,2))
        
        print("the predict is:%d,the real result is:%d" % (result,testLabel))
        if(testLabel!=result):
            error+=1
    print("the error rate is:%d" % (error/float(n)))
    

In [54]:
from os import listdir
#handwritingClassTest()